In [ ]:
from secrets import HADOOP_USER_NAME, SPARK_URI, HADOOP_NAMENODE

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F

In [ ]:
sc = SparkContext(SPARK_URI)
sparkSession = (
    SparkSession.builder.appName("example-pyspark-read-and-write1")
    .config("spark.hadoop.dfs.client.use.datanode.hostname", "true")
    .getOrCreate()
)

In [ ]:
# Read from hdfs
opusdata = sparkSession.read.csv(
    f"hdfs://{HADOOP_NAMENODE}:8020/raw/opusdata.csv", header=True, inferSchema=True
)
opusdata.show()

# sc = spark.sparkContext

In [ ]:
opusdata_filter_0 = opusdata.filter(opusdata["production_budget"] != 0)

In [ ]:
opusdata_filter_0 = opusdata_filter_0.filter(
    opusdata_filter_0["domestic_box_office"] != 0
)

In [ ]:
opusdata_filter_0 = opusdata_filter_0.filter(
    opusdata_filter_0["international_box_office"] != 0
)

In [ ]:
opusdata_dropped = opusdata_filter_0.drop(
    "movie_odid", "running_time", "production_method", "creative_type", "source"
)

In [ ]:
opusdata_years = opusdata_dropped.filter(opusdata_dropped["production_year"] >= 2010)

In [ ]:
opusdata_distinct = opusdata_years.dropDuplicates(["movie_name", "production_year"])

In [ ]:
opusdata_total_box_office = opusdata_distinct.withColumn(
    "total_box_office",
    opusdata_distinct["domestic_box_office"]
    + opusdata_distinct["international_box_office"],
).drop("domestic_box_office", "international_box_office")

In [ ]:
opusdata_droppped_na = opusdata_total_box_office.na.drop(subset=["genre", "sequel"])

In [ ]:
opusdata_droppped_na.coalesce(1).write.mode("overwrite").option('header',True).csv(
    f"hdfs://{HADOOP_NAMENODE}:8020/processed/opusdata_00.csv"
)